In [14]:
import pandas as pd
from collections import Counter
from datetime import datetime

In [15]:
data = pd.read_csv("../../gen/input/amazon.csv", sep = ";", header = None, usecols=range(16))


C:\Users\janva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
data.columns = ["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified"]

In [32]:
data

,asin_url,brand,model,link,product_title,name,rating,review_title,country,date,...,helpful,picture,verified,length,rating_float,date_trans,pronoun,variant,renewed,pronoun_length
0,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",wp,4.0,Works great.,the United States,"August 27, 2022",...,NaN,NaN,NaN,878,4,2022-08-27,5,black,yes,0.005695
1,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",J. R.,4.0,So far so good!,the United States,"September 5, 2022",...,NaN,NaN,NaN,618,4,2022-09-05,11,white,yes,0.017799
2,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Dov,4.0,Battery life is not the best but the phone is ...,the United States,"August 18, 2022",...,NaN,NaN,NaN,702,4,2022-08-18,5,black,yes,0.007123
3,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Jerome W,5.0,GREAT PRODUCT - IPhone 11,the United States,"August 26, 2022",...,NaN,NaN,NaN,603,5,2022-08-26,5,white,yes,0.008292
4,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Katie Sisemore,5.0,The phone worked good,the United States,"September 7, 2022",...,NaN,NaN,NaN,296,5,2022-09-07,2,purple,yes,0.006757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165470,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Shelly manning,1,this phone is only 3g. does NOT have 4g and mo...,the United States 🇺🇸,"October 6, 2021",...,none,no,yes,41,1,2021-10-06,0,orange,no,0.000000
165471,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Abubakar Jalloh,5,Very strong body.,the United States 🇺🇸,"February 4, 2020",...,1,no,yes,29,5,2020-02-04,1,orange,no,0.034483
165472,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Janet Decker,1,Phone not unlocked,the United States 🇺🇸,"April 19, 2021",...,none,no,yes,107,1,2021-04-19,0,orange,no,0.000000
165473,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Kelly D.,1,DO NOT ORDER THIS PHONE!,the United States 🇺🇸,"June 13, 2020",...,3,no,yes,117,1,2020-06-13,0,orange,no,0.000000


# Variable transformation

In [19]:
#to integer
integer_ratings = []
for index, row in data.iterrows():
    if row['rating'] == 'none':
        integer_ratings.append('none')
    else:
        integer_ratings.append(float(row['rating']))
data['rating_float'] = integer_ratings

In [20]:
#data['trans_date'] = 
date_trans = []
for row in data['date'].tolist():
    date_trans.append(datetime.strptime(row, "%B %d, %Y"))
    
data['date_trans'] = date_trans

# Length review

In [18]:
#length
number_occ = [len(str(m)) for m in data['review']]
data['length'] = number_occ

In [24]:
len(date_trans)

0

# Pronoun Usage

In [21]:
#pronouns
pronouns = ["I", "me", "my", "mine", "self", "myself"]
number_pronoun = []
for review in data['review']:
    occ = 0
    for word in str(review).split():
        if word in pronouns:
            occ += 1
    number_pronoun.append(occ)
data['pronoun'] = number_pronoun

# Colors
The following code looks which color belongs to which smartphone and groups them under secondary colors.

In [22]:
#colours
colors = ["blue", "green", "orange", "yellow", "purple", "brown", "gray","grey", "red", "black",
          "gold", "pink", "silver","white", "coral", "sage", "emerald", "glow", "bronze",
          "charcoal","violet", "graphite","burgundy", "cloud", "navy", "mint", "lavender",
          "phantom titanium", "olive", "peach", "night", "moon", "lime", "apricot", "starlight",
         "indigo", "sand", "rock", "sc490", "morning mist", "cyan", "winter mist", "mercury",
         "frosted", "cream", "electric shadow", "dark", "raspberry", "metallic", "blush"]
variant_colors = []

for index, row in data.iterrows():
    variant_color = []
    for color in colors:
        #Search for colors in the specs column
        if str(row['specs']).lower().find(color) != -1:
            variant_color.append(color)
        #If the word color is not found in specs, look in the title of the smartphone
        elif str(row['specs']).lower().find('color') == -1:
            if str(row['product_title']).lower().find(color) != -1:
                variant_color.append(color)           
    
    #If no color could be found, append none insteada
    if len(variant_color) == 0:
        variant_color.append('none')
    else:
        #do some corrections
        if variant_color[0] == "coral" or variant_color[0] == "peach" or variant_color[0] == "apricot":
            variant_color[0] = "orange"
        elif variant_color[0] == "sage" or variant_color[0] == "emerald" or variant_color[0] == "mint" or variant_color[0] == "olive"  or variant_color[0] == "lime":
            variant_color[0] = "green"
        elif variant_color[0] == "glow":
            variant_color[0] = "mixed"
        elif variant_color[0] == "bronze":
            variant_color[0] = "brown"
        elif variant_color[0] == "charcoal" or variant_color[0] == "graphite" or variant_color[0] == "phantom titanium" or variant_color[0] == "night" or variant_color[0] == "sc490" or variant_color[0] == "dark":
            variant_color[0] = "black"
        elif variant_color[0] == "violet" or variant_color[0] == "lavender" or variant_color[0] == "indigo":
            variant_color[0] = "purple"
        elif variant_color[0] == "burgundy":
            variant_color[0] = "red"
        elif variant_color[0] == "cloud" or variant_color[0] == "navy" or variant_color[0] == "winter mist" or variant_color[0] == "frosted":
            variant_color[0] = "blue"
        elif variant_color[0] == "grey" or variant_color[0] == "moon" or variant_color[0] == "rock" or variant_color[0] == "morning mist" or variant_color[0] == "electric shadow" or variant_color[0] == "metallic":
            variant_color[0] = "gray"
        elif variant_color[0] == "starlight" or variant_color[0] == "mercury":
            variant_color[0] = "silver" 
        elif variant_color[0] == "sand":
            variant_color[0] = "yellow" 
        elif variant_color[0] == "blush" or variant_color[0] == "raspberry":
            variant_color[0] = "pink" 
    #others category        
    variant_colors.append(variant_color[0])
data['variant'] = variant_colors

In [25]:
#Find missing colors
print(len(data[data['variant'] == 'none']))
data[data['variant'] == 'none']['specs'].unique()

5250


array(['none', 'Style: A10L( Support T-Mobile)',
       'Style: X6 Plus (Support Verizon, AT&T and T-Mobile)',
       'Style: Phone', 'Size: KeyboardStyle: Cynosa V2',
       'Size: KeyboardStyle: Cynosa Lite', 'Size: Phone',
       'Color: 12 packsSize: Small', 'Color: 12 packsSize: Large',
       'Style: Fully Unlocked (Renewed)', 'Style: T-mobile',
       'Capacity: Phone', 'Color: Titanium', 'Color: Granite',
       'Number of Items: 100', 'Number of Items: 200',
       'Style: Phone + SpeedTalk SIM Card', 'Color: 3 GB',
       'Color: Twilight', 'Style: Xperia Pro-I',
       'Color: 7 Transparent 18+256 GB',
       'Color: 7 Pro Transparent 16+512 GB', 'Color: US',
       'Color: Electric Auction', 'Style: Magnetic with Power',
       'Style: Universal', 'Style: Universal Gravity', 'Style: Magnetic',
       'Style: Base', 'Style: Universal with Power',
       'Color: U.S. regulations', 'Style: Boost Mobile', '[]',
       "['Color: Titanium']"], dtype=object)

# Renewed variable

In [24]:
#renewed variable ADD SPECS
renewed = []
for index, row in data.iterrows():
    if str(row['specs']).lower().find('renewed') != -1:
        renewed.append("yes")
    else:
        if str(row['product_title']).lower().find('renewed') != -1:
            renewed.append("yes")
        else:
            renewed.append("no")
data['renewed'] = renewed

# Pronoun / length variable

In [26]:
#pronoun divded by length
data['pronoun_length'] = data['pronoun']/data['length']

In [27]:
keepa_data = pd.read_excel("../../data/KeepaExport_all_variants.xlsx", usecols = "B:O")
#keepa_data.columns = ['keepa_title', 'review_count', 'oos90', 'price90', 'asin_url', 'asin', 'parent_asin', 'color']

In [37]:
data = pd.merge(data, keepa_data[["URL: Amazon", 'Parent ASIN']], left_on = "asin_url", right_on = "URL: Amazon", how = "left")

In [40]:
data

,asin_url,brand,model,link,product_title,name,rating,review_title,country,date,...,verified,length,rating_float,date_trans,pronoun,variant,renewed,pronoun_length,URL: Amazon,Parent ASIN
0,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",wp,4.0,Works great.,the United States,"August 27, 2022",...,NaN,878,4,2022-08-27,5,black,yes,0.005695,https://www.amazon.com/dp/B07ZPKN6YR,B097M74GSR
1,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",J. R.,4.0,So far so good!,the United States,"September 5, 2022",...,NaN,618,4,2022-09-05,11,white,yes,0.017799,https://www.amazon.com/dp/B07ZPKN6YR,B097M74GSR
2,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Dov,4.0,Battery life is not the best but the phone is ...,the United States,"August 18, 2022",...,NaN,702,4,2022-08-18,5,black,yes,0.007123,https://www.amazon.com/dp/B07ZPKN6YR,B097M74GSR
3,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Jerome W,5.0,GREAT PRODUCT - IPhone 11,the United States,"August 26, 2022",...,NaN,603,5,2022-08-26,5,white,yes,0.008292,https://www.amazon.com/dp/B07ZPKN6YR,B097M74GSR
4,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Katie Sisemore,5.0,The phone worked good,the United States,"September 7, 2022",...,NaN,296,5,2022-09-07,2,purple,yes,0.006757,https://www.amazon.com/dp/B07ZPKN6YR,B097M74GSR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165472,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Shelly manning,1,this phone is only 3g. does NOT have 4g and mo...,the United States 🇺🇸,"October 6, 2021",...,yes,41,1,2021-10-06,0,orange,no,0.000000,NaN,NaN
165473,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Abubakar Jalloh,5,Very strong body.,the United States 🇺🇸,"February 4, 2020",...,yes,29,5,2020-02-04,1,orange,no,0.034483,NaN,NaN
165474,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Janet Decker,1,Phone not unlocked,the United States 🇺🇸,"April 19, 2021",...,yes,107,1,2021-04-19,0,orange,no,0.000000,NaN,NaN
165475,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Kelly D.,1,DO NOT ORDER THIS PHONE!,the United States 🇺🇸,"June 13, 2020",...,yes,117,1,2020-06-13,0,orange,no,0.000000,NaN,NaN


# Version ASIN
The following code looks for the right asin (since the asins are not always labelled correctly because multiple asins are shown under each review page.

In [11]:
#Create list of dict that show the specs for each asin
specs_asin = []
for index, row in keepa_data.iterrows():
    joined_specs = []
    if str(row['Variation Attributes']) == 'nan':
        if str(row['Color']) == 'nan':
            joined_specs.append(None)
        else:
            joined_specs.append(row['Color'].replace(' ',''))
    else:
        for spec in row['Variation Attributes'].split(';'):
            #Add the specs that does not include the spec name itself (characterised by colon)
            joined_specs.append(''.join([x for x in spec.split() if x.find(':') == -1]))
    if str(row['Parent ASIN']) == 'nan':
        parent_asin = row['ASIN']
    else:
        parent_asin = row['Parent ASIN']
    specs_asin.append({parent_asin : {row['ASIN'] :list(filter(None, joined_specs))}})
specs_asin

[{'B0B6PV17MC': {'B0B63XWPWF': ['Charcoal+Seafoam', 'Pixel6a+Case']}},
 {'B0B6PV17MC': {'B0B3Q23BT4': ['Sage', 'Pixel6aonly']}},
 {'B0B6PV17MC': {'B0B3PSRHHN': ['Charcoal', 'Pixel6aonly']}},
 {'B0B6PV17MC': {'B0B63WLK7Q': ['Chalk+Charcoal', 'Pixel6a+Case']}},
 {'B0B6PV17MC': {'B0B63XWPWK': ['Chalk+Seafoam', 'Pixel6a+Case']}},
 {'B0B6PV17MC': {'B0B3PQPNKS': ['Chalk', 'Pixel6aonly']}},
 {'B0B6PV17MC': {'B0B63RMWPV': ['Sage+Seafoam', 'Pixel6a+Case']}},
 {'B0B6PV17MC': {'B0B63VYHJK': ['Charcoal+Charcoal', 'Pixel6a+Case']}},
 {'B0B6PV17MC': {'B0B63Z7JT4': ['Sage+Charcoal', 'Pixel6a+Case']}},
 {'B09PPWVQSB': {'B09BFTMQH9': ['Graphite', '128GB', 'S21FE', 'CellPhone']}},
 {'B09PPWVQSB': {'B08FYTSXGQ': ['CloudNavy', '128GB', 'S20FE', 'CellPhone']}},
 {'B09PPWVQSB': {'B09BFRV59N': ['White', '128GB', 'S21FE', 'CellPhone']}},
 {'B09PPWVQSB': {'B09BFSCCNB': ['Graphite', '256GB', 'S21FE', 'CellPhone']}},
 {'B09PPWVQSB': {'B08FYTRF6J': ['CloudLavender',
    '128GB',
    'S20FE',
    'CellPhone']}},
 

In [48]:
for spec in specs_asin:
    for key, value in spec.items():
        if key == "B09J3G4TVZ":
            print(value)

{'B07T94DM3Y': ['128GB', 'PrismWhite', 'AT&T', 'Renewed']}
{'B07YX77QR1': ['512GB', 'PrismBlue', 'AT&T', 'Renewed']}
{'B07SVG5HFL': ['512GB', 'PrismWhite', 'Verizon', 'Renewed']}
{'B086TQQ2M2': ['128GB', 'PrismWhite', 'GSMCarriers', 'Renewed']}
{'B07YBPDWHC': ['128GB', 'PrismBlue', 'T-Mobile', 'Renewed']}
{'B082WJ9BR8': ['512GB', 'PrismBlack', 'AT&T', 'Renewed']}
{'B07SQB3LMB': ['128GB', 'FlamingoPink', 'Verizon', 'Renewed']}
{'B07PGQFGBW': ['512GB', 'PrismWhite', 'Unlocked', 'Renewed']}
{'B07STDQDKF': ['128GB', 'PrismWhite', 'Verizon', 'Renewed']}
{'B07WK8HVW1': ['128GB', 'FlamingoPink', 'AT&T', 'Renewed']}
{'B07YNTQJNW': ['512GB', 'PrismWhite', 'AT&T', 'Renewed']}
{'B07T94Q9WW': ['128GB', 'PrismBlue', 'AT&T', 'Renewed']}
{'B07PFL1NL9': ['512GB', 'PrismBlue', 'Unlocked', 'Renewed']}
{'B07T8CN8WZ': ['128GB', 'PrismBlack', 'T-Mobile', 'Renewed']}
{'B07PDN414K': ['128GB', 'PrismWhite', 'Unlocked', 'Renewed']}
{'B086T6YZJ3': ['128GB', 'PrismBlack', 'GSMCarriers', 'Renewed']}
{'B07SQB29JS'

In [29]:
def retrieve_asin(specs_asin, spec, parent_asin):
    child_asins = []
    asin_match = None
    #First retrieve all children belonging to parent asin
    for specs in specs_asin:
        for key, value in specs.items():
            if key == parent_asin:
                child_asins.append(value)
            else:
                pass
    #If there's only one child, it means that every review is written about that phone
    if len(child_asins) == 1:
        for key, value in child_asins[0].items():
            asin_match = key
    else:  
        #For every child belonging to that parent
        for child in child_asins:
            #Get the asin and specs
            for child_asin, child_specs in child.items():
                #loop through specs
                for child_spec in child_specs:
                    #If spec doesn't match, break and move to the next child asin
                    if child_spec.lower() not in spec.lower():
                        break
                    #if this was the last spec and hasn't broken yet, return child
                    elif child_specs.index(child_spec) == len(child_specs) -1:   
                        asin_match = child_asin
    return asin_match

In [42]:
version_asin = []
i = 1
for index, row in data.iterrows():
    spec = ''.join(row['specs'].split())
    #If parent asin is missing
    if str(row['Parent ASIN']) == 'nan':
        #retrieve asin from link
        asin_from_link = row['asin_url'].replace("https://www.amazon.com/dp/", "")
        try:
            #look for parent asin in keepa database
            parent_asin = keepa_data[keepa_data['ASIN']==asin_from_link]['Parent ASIN'].iloc[0]
        except:
            #if it's missing, it means that asin is the parent
            parent_asin = asin_from_link
    else:
        #if parent is present, use the parent
        parent_asin = row['Parent ASIN']
    version_asin.append(retrieve_asin(specs_asin, spec, parent_asin))
    i += 1
    if i%1000 == 0:
        print(f"{i}/{len(data)}")
data['version_asin'] = version_asin

1000/165477
2000/165477
3000/165477
4000/165477
5000/165477
6000/165477
7000/165477
8000/165477
9000/165477
10000/165477
11000/165477
12000/165477
13000/165477
14000/165477
15000/165477
16000/165477
17000/165477
18000/165477
19000/165477
20000/165477
21000/165477
22000/165477
23000/165477
24000/165477
25000/165477
26000/165477
27000/165477
28000/165477
29000/165477
30000/165477
31000/165477
32000/165477
33000/165477
34000/165477
35000/165477
36000/165477
37000/165477
38000/165477
39000/165477
40000/165477
41000/165477
42000/165477
43000/165477
44000/165477
45000/165477
46000/165477
47000/165477
48000/165477
49000/165477
50000/165477
51000/165477
52000/165477
53000/165477
54000/165477
55000/165477
56000/165477
57000/165477
58000/165477
59000/165477
60000/165477
61000/165477
62000/165477
63000/165477
64000/165477
65000/165477
66000/165477
67000/165477
68000/165477
69000/165477
70000/165477
71000/165477
72000/165477
73000/165477
74000/165477
75000/165477
76000/165477
77000/165477
78000/16

# Brands
Correction to the other brand column since that sometimes shows the provider etc.

In [ ]:
brands = ['apple', 'iphone', 'google', 'samsung', 'motorola', 'tracfone',
          'oneplus', 'tcl', 'blu ', 'jitterbug', 'zte', 'huawei',
          'nokia','sony', 'lg', 'htc', 'lenovo', 'xiaomi',
         'honor', 'oppo', 'realme', 'vivo', 'meizu', 'blackberry',
         'asus', 'alcatel', 'microsoft', 'energizer', 'cat', 'sharp',
         'ulefone', 'panasonic', 'nuu', 'oukitel', 'blackview', 'xgody']
all_brands = []
i = 0
for index, row in data.iterrows():
    smartphone_brand = []
    for brand in brands:
        if str(row['brand']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
        elif str(row['product_title']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
        elif str(row['model']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
        elif str(row['specs']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
    if len(smartphone_brand) == 0:
        smartphone_brand.append(None)
    elif smartphone_brand[0] == 'iphone':
        smartphone_brand[0] = 'apple'
    all_brands.append(smartphone_brand[0])
    if i%10000 == 0:
        print(f"{round(i/len(data),3)*100}%")
    i += 1
data['brand_overall'] = all_brands

# Brand mentions in reviews
Adding a column with the number of times a brand was mentioned in the review

In [ ]:
all_mentions = []
for index, row in data.iterrows():
    if row['brand_overall'] == "apple":
        mention = (str(row['review']).count(str(row['brand_overall']))+str(row['review']).count("iphone"))/len(str(row['review']).split())
    else:
        mention = str(row['review']).count(str(row['brand_overall']))/len(str(row['review']).split())
    all_mentions.append(mention)
data['brand_mention'] = all_mentions

# Iphone variable

In [ ]:
iphone = []
for index, row in data.iterrows():
    if row['brand'].strip() == 'Apple':
        iphone.append(1)
    elif 'apple' in row['product_title'].lower():
        iphone.append(1)
    elif 'iphone' in row['product_title'].lower():
        iphone.append(1)
    else:
        iphone.append(0)
data['iphone'] = iphone

# Spec mention variable
A variable showing how often the reviewer talks about specs in a review as a percentage

In [ ]:
most_imp_specs = ['battery', 'processor', 'ram', 'screen', '3g', '4g', '5g',
                  'memory', 'camera', 'cpu', 'ppi', 'dppi', 'resolution']
spec_mentions = []
for review in data['review']:
    if str(review) == 'nan':
        spec_men = 0
    else:
        review = review.lower()
        spec_men = (len([word for word in review.split() if word.lower() in most_imp_specs])+
                    len(re.findall("([0-9]+)?[tgmk]b", review))+
                   len(re.findall("[0-9]+x[0-9]+", review)))/len(review.split())
    spec_mentions.append(spec_men)
data['spec_mention'] = spec_mentions

In [46]:
data[data['version_asin'].isna()].to_csv("../../gen/audit/Amazon_missing.csv", sep = ";")

In [29]:
data_merged.to_csv("../../gen/temp/Amazon_reviews_clean_0_1000.csv", sep = ";")